In [2]:
from modified_vasp import write_vasp_input_files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import uuid
import ase.io
from tqdm import tqdm
import os
from os import path
from glob import iglob
import warnings
warnings.filterwarnings("ignore")

In [3]:
with open("mapping_adslab_slab.pkl", "rb") as input_file:
    slab_adslab_mappings = pickle.load(input_file)
    ## get makkings of sid (a random number) to systems (adslabs) that are in the OC20 dataset 
with open("oc20_data_mapping.pkl", "rb") as input_file:
    mappings = pickle.load(input_file)

In [4]:
## Adeesh data from the perspecive paper
df = pd.read_csv("gemnet-relaxations.csv", index_col=0)
df["error"] = np.abs(df["energy dE [eV]"] - df["genmet_T_predE"])
# sort error column in descending order
df.sort_values(by=['error'], inplace=True, ascending=False)

In [5]:
## pick the 10 nonmetallic systems with the highest ML errors
df_nonmetals = df.query("material_type == 'nonmetals'")
sids = df_nonmetals.iloc[0:11]["ID"].values

In [6]:
## load the mappings stored from previous experiments
with open('dft_settings_study_mappings.pkl', 'rb') as read_file:
    updated_mappings = pickle.load(read_file)
# updated_mappings = updated_mappings.to_dict()

In [7]:
## Generate the vasp input files
experiments_mappings = {}
# loop over the randomly selected systems and generate vasp input files while saving the calcualtion meta data
for sid in tqdm(sids):
    system_mapping = mappings[sid]
    slab_sid = slab_adslab_mappings[sid]
    slab_atoms = ase.io.read(f"/home/jovyan/shared-datasets/OC20/trajs/slabs/trajectories_all_02_05/{slab_sid}.traj", 0)
    
    for kpts in [40]:
            ## define the vasp flags for any modifications to the vasp inputs
        VASP_FLAGS = {
            "encut": 350,
            "kpts_multiplier": kpts,
            "sigma": 0.2,
            "ispin": 1, 
            "ismear": 1,
            "ibrion": 2,
            "nsw": 0,
            "isif": 0,
            "isym": 0,
            "lreal": "Auto",
            "ediffg": -0.03,
            "symprec": 1e-10,
            "ncore": 4,
            "gga": "RP",
            "pp": "PBE",
            "xc": "PBE",
            "lcharg":False,
            "lwave":False,
            "laechg": False,
        }

        # create a random number for the experiment uuid
        experiment_uuid = uuid.uuid4().hex

        # save meta data about the system and the calculation
        # put None for adsorbate information since this is just a slab calculation
        meta_data= {
            'bulk_id':system_mapping["bulk_id"],
            "ads_id": system_mapping["ads_id"],
            "bulk_mpid": system_mapping["bulk_mpid"], 
            "bulk_symbols": system_mapping["bulk_symbols"], 
            "ads_symbols": None,
            "miller_index": list(system_mapping["miller_index"]),
            "shift": system_mapping["shift"],
            "top": system_mapping["top"],
            "class": system_mapping["class"],
            "anomaly": system_mapping["anomaly"],
            "atoms": slab_atoms,
            "encut": VASP_FLAGS["encut"],
            "k_point_multiplier": VASP_FLAGS["kpts_multiplier"], 
            "sigma": VASP_FLAGS["sigma"],
            "ispin": VASP_FLAGS["ispin"],
            "ismear": VASP_FLAGS["ismear"],
            "xc": VASP_FLAGS["xc"],
            "oc20_sid": slab_sid,
            "frame_num":"initial",
            "final_energy":None,
            "unconstrained_atoms_forces":None,
            "nelm":None,
            "last_scf_cycle_dE": None,
        }
        if meta_data in updated_mappings.values():
            print("Experiment already exists")
        else:
            experiments_mappings[experiment_uuid] = meta_data
            #storing the vasp input files in a folder with the experiment uuid
            os.mkdir("kpts_error_cancel_study/{x}".format(x = experiment_uuid))
            write_vasp_input_files(slab_atoms, vasp_flags =VASP_FLAGS, outdir = "kpts_error_cancel_study/{x}".format(x = experiment_uuid))

100%|██████████| 11/11 [00:00<00:00, 32.12it/s]


In [9]:
# update the mappings dictionary with the new experiments
updated_mappings.update(experiments_mappings)
# ## storing the meta-data in a json file
with open('dft_settings_study_mappings.pkl', 'wb') as fout:
    pickle.dump(updated_mappings, fout)